# API KEY

In [1]:
# Simpan API Key OpenRouter kamu
OPENROUTER_API_KEY = "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"  # <-- Ganti dengan punyamu

# Library

In [2]:
import json
import os
import faiss
import numpy as np
import requests
from sentence_transformers import SentenceTransformer
from gtts import gTTS
from moviepy import TextClip, AudioFileClip, CompositeVideoClip

In [3]:
# Load FAISS Index
index = faiss.read_index("output/faiss_index.bin")

# Load Mapping sumber chunk
with open("output/chunk_mapping.json", "r", encoding="utf-8") as f:
    chunk_sources = json.load(f)

# Load ulang semua chunk text
with open("output/cleaned_text/epub_chunks_translated.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Gabungkan semua chunk
all_chunks = []
for book_title, chunks in data.items():
    for chunk in chunks:
        if chunk.strip():
            all_chunks.append(chunk)

# Load model embedding
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


In [4]:
def search_best_chunk(question, top_k=1):
    """Cari chunk paling relevan untuk pertanyaan."""
    question_embedding = model.encode([question]).astype('float32')
    D, I = index.search(question_embedding, top_k)

    results = []
    for idx in I[0]:
        if idx < len(all_chunks):
            results.append((chunk_sources[idx], all_chunks[idx]))
    return results


In [5]:
import requests

def summarize_with_deepseek(text, api_key=OPENROUTER_API_KEY):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    prompt = f"Summarize this text into a short, clear, concise answer suitable for a 1-minute video narration:\n\n{text}"

    data = {
        "model": "tngtech/deepseek-r1t-chimera:free",  # Updated model ID
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.5
    }

    response = requests.post(url, headers=headers, json=data)

    # Check if there is an error in the response
    if response.status_code != 200:
        print(f"❌ ERROR: {response.status_code} - {response.text}")
        return "[SUMMARY FAILED]"

    response_data = response.json()

    if 'choices' not in response_data:
        print(f"❌ ERROR: {response_data}")
        return "[SUMMARY FAILED]"

    summary = response_data['choices'][0]['message']['content']
    return summary


In [6]:
def text_to_speech(text, output_audio_path="output/answer_audio.mp3"):
    tts = gTTS(text=text, lang='en')
    tts.save(output_audio_path)
    return output_audio_path


In [7]:
def create_video(text, audio_path, output_video_path="output/answer_video.mp4"):
    # Buat TextClip dengan parameter yang benar
    text_clip = TextClip(
        text, 
        fontsize=70,
        color='white',
        size=(1280, 720),
        bg_color='black',
        font="Arial-Bold",
        method='label'
    )
    
    # Set durasi sesuai audio
    audio_clip = AudioFileClip(audio_path)
    text_clip = text_clip.set_duration(audio_clip.duration)
    text_clip = text_clip.set_audio(audio_clip)
    
    # Tulis video
    final_video = CompositeVideoClip([text_clip])
    final_video.write_videofile(
        output_video_path, 
        fps=24,
        codec='libx264',
        audio_codec='aac'
    )
    
    return output_video_path

In [8]:
# Pertanyaan user
question = "What does Islam say about the importance of youth?"

# Step 1: Cari Chunk
results = search_best_chunk(question, top_k=1)
source, answer = results[0]

print(f"📖 Source: {source}")
print(f"📝 Original Answer: {answer}")

# Step 2: Summarize dengan DeepSeek
summarized_answer = summarize_with_deepseek(answer)
print(f"📝 Summarized Answer: {summarized_answer}")

# Step 3: Buat Audio
audio_path = text_to_speech(summarized_answer)



📖 Source: الأربعون الشبابية - Chunk 1
📝 Original Answer: Cover forty youth Book: Forty youth, author: Muhammad Khair Ramadan Yusuf, unknown, forty youth, Muhammad Khair Ramadan Muharram 1438 AH. Source: The Golden Comprehensive is anonymous in the name of God, the Most Gracious, the Most Merciful. And educate them to strengthen, and direct them to advocacy and jihad, such as Ali, Musab, Moaz, Ibn Masoud, Ibn Omar, and others, may God be pleased with them all. And their news is many in the Sunnah and the generous prophetic biography, and if the word (youth) and (the boys) did not mention and what they behave from them, they were the audience of the new religion and its fuel, and the movement of society and its activity and vitality, and the Messenger of God, peace and blessings of God be upon him, assigned them to great deeds, to feel the responsibility of the religion About him with senior leaders. I loved the collection of a group of hadiths in which the young and young people are men